In [2]:
from collections import deque
import pickle

def get_C(tree, labels, v, h):
    neighbors = get_N(tree, v, h)
    return {labels[u] for u in neighbors}

def get_N(tree, v, h):
    visited = set()
    queue = [(v, 0)]
    
    while queue:
        node, depth = queue.pop(0)
        if node not in visited:
            visited.add(node)
            if depth < h:
                for neighbor in tree[node]:
                    queue.append((neighbor, depth + 1))
    return visited

def bfs_shortest_paths(tree, start):
    n = len(tree)
    distances = [-1] * n
    distances[start] = 0
    queue = deque([start])

    while queue:
        node = queue.popleft()
        for neighbor in tree[node]:
            if distances[neighbor] == -1:
                distances[neighbor] = distances[node] + 1
                queue.append(neighbor)

    return distances

def compute_r_m_values(tree, labels, k):
    r_values = []
    m_values = []

    for v in range(len(tree)):
        r, m = 0, 0

        # Find the smallest h for which |C(v, h)| = k (r(v))
        for h in range(len(tree)):
            if len(get_C(tree, labels, v, h)) == k:
                r = h
                break

        # Find the smallest h for which |N(v, h)| >= k (m(v))
        for h in range(len(tree)):
            if len(get_N(tree, v, h)) >= k:
                m = h
                break

        r_values.append(r)
        m_values.append(max(1, m))  # Ensure m(v) is never zero

    return r_values, m_values

def node_labeling_optimization(tree, k):
    n = len(tree)
    labels = [0] * n
    root = 0
    max_depth = max(bfs_shortest_paths(tree, root))

    def dfs(node, parent, depth, assigned_labels):
        if depth == 0:
            for label in range(k):
                if label not in assigned_labels:
                    labels[node] = label
                    assigned_labels.add(label)
                    break
        else:
            available_labels = set(range(k)) - assigned_labels
            for neighbor in tree[node]:
                if neighbor != parent:
                    for label in available_labels:
                        labels[neighbor] = label
                        available_labels.remove(label)
                        break
                    dfs(neighbor, node, depth - 1, assigned_labels | {labels[neighbor]})

    dfs(root, -1, max_depth, set())

    return labels

tree_list = pickle.load(open("Small_Examples_of_AdjLists_of_Trees", "rb"))
k_values = pickle.load(open("Small_Examples_of_k_values", "rb"))

max_ratios = []

for i, (tree, k) in enumerate(zip(tree_list, k_values)):
    labels = node_labeling_optimization(tree, k)
    r_values, m_values = compute_r_m_values(tree, labels, k)
    print(f"Labels: {labels}")
    ratios = [r_values[v] / m_values[v] for v in range(len(tree))]
    #print(f"r(v)/m(v) ratios: {ratios}")
    max_ratios.append(max(ratios))
    #print(max_ratios)
    #print(max_ratios)
print(f"max ratio:{max(max_ratios)}")


Labels: [0, 0, 1, 2, 3, 1, 2, 3, 4, 5, 0, 2, 3, 4, 0, 1, 3, 4, 5, 0, 1, 2, 4, 2, 3, 4, 5, 0, 1, 1, 2, 3, 1, 2, 3, 0, 3, 4, 0, 2, 0, 2, 3, 1, 0, 3, 0, 1, 3, 5, 0, 1, 3, 4, 1, 0, 2, 0, 1, 4, 5, 0, 4, 3, 3, 4, 2]
Labels: [0, 0, 1, 2, 1, 2, 0, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 1, 3, 4, 5, 6, 7, 2, 3, 4, 5, 6, 0, 3, 0, 2, 4, 5, 6, 7, 8, 0, 2, 3, 4, 1, 3, 0, 3, 4, 5, 6, 4, 2]
Labels: [0, 0, 1, 2, 0, 1, 0, 2, 0, 0, 0, 1, 1, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0]
Labels: [0, 0, 1, 2, 3, 3, 2, 4, 5, 4, 5, 6, 2, 5, 2, 4, 4, 4, 5, 7, 2, 2, 6, 4, 7, 4, 5, 6, 6, 2, 7, 7, 0, 4, 0, 0, 4, 7, 0, 0, 7, 7, 0, 2, 0, 0, 0, 0, 7, 0, 0, 0, 4, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 7, 7, 0, 0]
Labels: [0, 0, 1, 2, 3, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 1, 2, 3, 2, 2, 3